In [1]:
ls

7q4b_t1_protein_only.xtc  em_protien_only.gro       stacking_contacts.ipynb


In [7]:
import numpy as np
import matplotlib.pyplot as plt
import mdtraj as md
import itertools


def stacking_distances(traj,topology, protein_chainA_residues,protein_chainB_residues,stack_number,file_name):
    t = md.load(traj,top=topology)
    stack_1 = (protein_chainA_residues+protein_chainB_residues)*stack_number 
    stack_2 =stack_1+protein_chainA_residues+protein_chainB_residues
    if stack_number == 1:
    	group1 = np.arange(0,stack_1)
    else:
        group1 = np.arange(stack_1/2,stack_1)
    group2 = np.arange(stack_1,stack_2)
    pairs = list(itertools.product(group1,group2))
    contacts = md.compute_contacts(t,pairs,scheme='closest-heavy')

    file =file_name
    write_mode= "w"
    _out = open(file,write_mode)

    for i in range(len(pairs)):
        if i < len(pairs)-1:
            _out.write(str(pairs[i][0])+"-"+str(pairs[i][1])+",")
        else:
            _out.write(str(pairs[i][0])+"-"+str(pairs[i][1]))
    _out.write("\n")

    for i in range(len(contacts[0])):
        for j in range(len(contacts[0][0])):
            if j < len(contacts[0][0])-1:
                _out.write(str(contacts[0][i][j])+',')
            else:
                _out.write(str(contacts[0][i][j]))
        _out.write("\n")

    _out.close()


def contacts(distance,i = 0):
    contacts=[]
    for j in range(len(distance[0])):
        if (distance[i][j] <= .45):
            contacts.append(1)
        else:
            contacts.append(0)
    
    number_contact = sum(contacts)
    return number_contact

def contacts_over_time(distances):
    distance = np.loadtxt(distances,delimiter=",",skiprows=1)
    over_time = []
    for i in range(len(distance)):
        total = contacts(distance,i)
        over_time.append(total)
    return over_time

def native_contacts(data):
    distance = np.loadtxt(data,delimiter=',',skiprows=1)
    over_time=[]
    contacts=[]
    native_columns = []
    for j in range(len(distance[0][:])):
        if distance[0][j] <= 0.45:
            native_columns.append(j)

    for i in range(len(distance)):
        for j in native_columns:
            if (distance[i][j] <= .45):
                contacts.append(1)
            else:
                contacts.append(0)

        number_contact = sum(contacts)
        over_time.append(number_contact)
        contacts=[]
    return over_time

trajectory = '7q4b_t1_protein_only.xtc'#input("Please enter trajectory file name: ")
topology = '../7q4b_fibril.gro' #input("Please enter topology file name: ")
protein_chainA_residues = 42 #int(input("Please enter number of protein residues: "))
protein_chainB_residues = 42 #int(input("Please enter number of ligand residues: "))
stack_number = int(input("Please enter stack number: "))
file_name = '7q4b_t1_'+str(stack_number)+'_stack_distances.csv'#input("Please enter output file name: ")
file_name2 = '7q4b_t1_'+str(stack_number)+'_stack_contacts.csv' #input("Please enter name of second output file: ")
file_name3 = '7q4b_t1_'+str(stack_number)+'_stack_native_contacts.csv' #input("Please enter name of third (native) output file: ")



stacking_distances(trajectory,topology, protein_chainA_residues,protein_chainB_residues,stack_number,file_name)

x = contacts_over_time(file_name)
time_scale = 200

file = open(file_name2,"w")

for i in range(len(x)):
    file.write(str(i*time_scale/1000)+","+str(x[i]))
    file.write("\n")

file.close()

native = native_contacts(file_name)

file = open(file_name3,"w")

for i in range(len(native)):
    file.write(str(i*time_scale/1000)+","+str(native[i]))
    file.write("\n")

file.close()


Please enter stack number:  4
